## Extractive - Centroid base Text summarization

* This code is an implementation of the paper **"Centroid-based Text Summarization through Compositionality of Word Embeddings"**


In [1]:
import base
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


class CentroidBOWSummarizer(base.BaseSummarizer):

    def __init__(self,
                 language='english',
                 preprocess_type='nltk',
                 stopwords_remove=True,
                 length_limit=10,
                 debug=False,
                 topic_threshold=0.3,
                 sim_threshold=0.95):
        super().__init__(language, preprocess_type, stopwords_remove, length_limit, debug)
        self.topic_threshold = topic_threshold
        self.sim_threshold = sim_threshold
        return

    def summarize(self, text, limit_type='word', limit=100):
        raw_sentences = self.sent_tokenize(text)
        clean_sentences = self.preprocess_text(text)

        vectorizer = CountVectorizer()
        sent_word_matrix = vectorizer.fit_transform(clean_sentences)

        transformer = TfidfTransformer(norm=None, sublinear_tf=False, smooth_idf=False)
        tfidf = transformer.fit_transform(sent_word_matrix)
        tfidf = tfidf.toarray()

        centroid_vector = tfidf.sum(0)
        centroid_vector = np.divide(centroid_vector, centroid_vector.max())
        for i in range(centroid_vector.shape[0]):
            if centroid_vector[i] <= self.topic_threshold:
                centroid_vector[i] = 0

        sentences_scores = []
        for i in range(tfidf.shape[0]):
            score = base.similarity(tfidf[i, :], centroid_vector)
            sentences_scores.append((i, raw_sentences[i], score, tfidf[i, :]))

        sentence_scores_sort = sorted(sentences_scores, key=lambda el: el[2], reverse=True)

        count = 0
        sentences_summary = []
        for s in sentence_scores_sort:
            if count > limit:
                break
            include_flag = True
            for ps in sentences_summary:
                sim = base.similarity(s[3], ps[3])
                # print(s[0], ps[0], sim)
                if sim > self.sim_threshold:
                    include_flag = False
            if include_flag:
                # print(s[0], s[1])
                sentences_summary.append(s)
                if limit_type == 'word':
                    count += len(s[1].split())
                else:
                    count += len(s[1])

        summary = "\n".join([s[1] for s in sentences_summary])
        return summary

In [2]:
viet_in = '''Khởi_tố kẻ_trộm hơn 1 tạ thóc và hơn 8 triệu đồng của chú ruột để lấy tiền mua ma_tuý

Ngày 27/3 , Cơ_quan Cảnh_sát điều_tra Công_an TP. Hưng_Yên , tỉnh Hưng_Yên cho biết , đơn_vị vừa ra quyết_định khởi_tố vụ án , khởi_tố bị_can đối_với đối_tượng Mai_Văn_Thương ( SN 1989 , trú tại đội 11 , thôn An_Chiểu 1 , xã Liên_Phương , TP. Hưng_Yên ) để điều_tra về hành_vi trộm_cắp tài_sản .
Theo tài_liệu điều_tra của cơ_quan công_an , vào_khoảng 7h30 ngày 13/3 , lợi_dụng gia_đình ông Mai_Văn_Thịnh ( chú ruột đối_tượng Thương ) ở cạnh nhà đi vắng , đối_tượng này đã đạp gãy chấn_song cửa_sổ , đột_nhập vào nhà ông Thịnh trộm_cắp 121kg thóc mang bán cho người cùng thôn lấy 700.000 đ .
Không dừng lại , sau đó đối_tượng tiếp_tục quay lại lục_soát tủ nhà ông Thịnh trộm_cắp 8.500.000 đ tiền_mặt ( ông Thịnh để dưới đáy tủ ) , rồi dùng số tiền trên để đi mua ma_tuý về sử_dụng và tiêu_xài hết 6.080.000 đ .
Đến ngày 15/3 , đối_tượng Thương đã đến Cơ_quan điều_tra Công_an TP. Hưng_Yên tự_thú và khai nhận toàn_bộ hành_vi phạm_tội của mình , đồng_thời giao_nộp cho cơ_quan công_an 3.120.000 đ .
Hiện Công_an TP. Hưng_Yên đã thu_giữ toàn_bộ 121kg thóc đối_tượng đã trộm_cắp để trao_trả cho gia_đình ông Thịnh .
Được biết Thương là đối_tượng nghiện ma_tuý từ nhiều năm nay , đã có 1 tiền_án về tội Tàng_trữ trái_phép chất ma_tuý bị TAND tỉnh Hưng_Yên xử_phạt 2 năm 3 tháng tù_giam .
Ra tù năm 2016 , đối_tượng này tiếp_tục có hành_vi cố_ý gây thương_tích , bị Công_an TP. Hưng_Yên ra quyết_định xử_phạt 2,5 triệu đồng .
Vụ án đang được Công_an TP. Hưng_Yên hoàn_thiện hồ_sơ để xử_lý Mai_Văn_Thương theo quy_định của pháp_luật .

Đối_tượng Mai_Văn_Thương tại cơ_quan công_an .'''

eng_in = '''The young boy who was hunting rabbits in the forest was not sure whether it had been the woman’s last cry or the child’s first that alerted him. He turned suddenly, sensing the possible danger, his eyes searching for an animal that was so obviously in pain. He had never known any animal to scream in quite that way before. He edged towards the noise cautiously; the scream had now turned to a whine but it still did not sound like any animal he knew. He hoped it would be small enough to kill; at least that would make a change from rabbit for dinner. The young boy moved stealthily towards the river, where the strange noise came from, running from tree to tree, feeling the protection of the bark against his shoulder blades, something to touch. Never stay in the open, his father had taught him. When he reached the edge of the forest, he had a clear line of vision all the way down the valley to the river, and even then it took him some time to realise that the strange cry emanated from no ordinary animal. He continued to creel) towards the whining, but he was out in the open on his own now. Then suddenly he saw the woman, with her dress above her waist, her bare legs splayed wide apart. He had never seen a woman like that before. He ran quickly to her side and stared down at her belly, quite frightened to touch. There, lying between the woman’s legs, was the body of a small, damp, pink animal, attached by I’l something that looked like rope. The young hunter dropped his freshly skinned rabbits and collapsed on his knees beside the little creature. He gazed for a long, stunned moment and then turned his eyes towards the woman, immediately regretting the decision. She was already blue with cold; her tired twentythree-year-old face looked middle-aged to the boy; he did not need to be told that she was dead. He picked up the slippery little body – had you asked him why, and no one ever did, he would have told you that the tiny fingernails clawing the crumpled face had worried him – and then he became aware that mother and child were inseparable because of the slimy rope. He had watched the birth of a lamb a few days earlier and he tried to remember. Yes, that’s what the shepherd had done, but dare he, with a child? The whining had stopped and he sensed that a decision was now urgent. He unsheathed his knife, the one he had skinned the rabbits with, wiped it on his sleeve and hesitating only for a moment, cut the rope close to the child’s body. Blood flowed freely from the severed ends. Then what had the shepherd done when the lamb was born? He had tied a knot to stop the blood. Of course, of course; he pulled some grass out of the earth beside him and hastily tied a crude knot in the cord. Then he took the child in his arms. He rose slowly from his knees, leaving behind him three dead rabbits and a dead woman who had given birth to this child. Before finally turning his back on the mother, he put her legs together, and pulled her dress down over her knees. It seemed to be the right thing to do. ‘Holy God,’ he said aloud, the first thing he always said when he had done something very good or very bad. He wasn’t yet sure which this was.'''

viet_in_2 = '''Hồi còn nhỏ, ngây ngô nghĩ rằng, một mối quan hệ, nếu bị khoảng cách chia xa, thể nào cũng sẽ tàn phai. Thế nhưng nếu không xa nhau, thì tình cảm cũng sẽ còn mãi. Thế là hồi đó, chia tay bạn bè cấp 2, tôi buồn hết 2 năm cấp 3. Cảm giác của một đứa không có gì bám víu giữa thành phố xa lạ nhưng lại không chịu chấp nhận một ai xung quanh. Riết rồi khó chia sẻ đến tận bây giờ. Vốn tôi nghĩ, thật tình những vấn đề của bản thân, chẳng ai hiểu nổi. Một đứa trẻ 15 tuổi, bảo là vấn đề to lớn như phá sản, tang tóc,... thật không phải. Nhưng vấn đề có nhỏ không? Đáp án là không! Thay đổi môi trường sống thật ra chỉ là giọt nước tràn li cho rất nhiều những vấn đề khác nhau, khó có lối ra, chỉ toàn nút thắt. Có chăng chỉ là giống sợi chỉ đỏ, khâu mà có nút chỗ nào, thì đành cắt đi khâu lại ở nút đó. Cứ thế bản thân vừa muốn kết nối, nhưng cũng tạo cho mình một rào cản khó có thể xóa bỏ. Nói là chia sẻ, nhưng chỉ dừng lại ở mặt chữ, là chia sẻ câu chuyện, trên nền chất liệu phong cách ngôn ngữ sinh hoạt, chứ lời nói chẳng có tí cảm xúc nào. Riết thành quen, bây giờ lời nói buông ra, cũng chẳng còn biết làm thế nào cho đằm thắm thứ cảm xúc đang bồn chồn muốn bộc phát trong tim. 
Lớn lên một chút, thôi thì môi trương thay đổi nhiều, chí ít đã quen những biến thiên hàng ngày của cuộc sống. Sống mà không có sự thay đổi, thì còn gọi gì là cuộc sống. Mãi bình an vui vẻ, chỉ có trên thiên đàng, mãi đau khổ sợ hãi, chỉ ở dưới địa ngục. Ba tầng trên thế gian, chắc nhân thế là hỗn loạn nhất! Bao nhiêu hỉ nộ ái ố đều phải kinh qua thì mới là người. Thế nhưng có lẽ cũng vì thế mà tôi vỡ òa, thật ra con người ta ở cạnh nhau mà lòng vẫn xa nhau được đấy thôi! Cái gọi là "cô đơn trong chính mối quan hệ của mình" thì giữa người nào cũng có. Người ta kết nối về mặt địa lí thì dễ dàng, chứ kết nối về mặt tâm hồn, e khó! Những con người của hôm nay, chúng tôi đối mặt nhau, ngồi cạnh nhau mà ngỡ là "một vòng trái đất". Người kia nghĩ gì, tôi không biết, cũng dần cảm thấy không còn muốn biết. Thành ra hai bên tự xây tường thành bao bọc cảm xúc. Bây giờ mình ôm nhau một cái được không? Để chất dẫn truyền thần kinh sinh ra làm ta hưng phấn? Tôi lại một lần nữa e khó! 
Bất cứ mối quan hệ nào, tôi cho rằng đều giống như 2 người đang đi trên dây. Khi người kia muốn tiến đến thì người còn lại phải đứng im và ngược lại, thành ra cứ mãi chệch nhịp. Còn nếu cả 2 cùng di chuyển được, thì phải có một cái gọi là "ĐỒNG TÂM". Suy nghĩ và cảm xúc cùng đồng điệu, chứ không cả 2 ngã nhào ngay! 
Nói chung suy đi xét lại, bí quyết để duy trì mối quan hệ tốt là gì? Không có quy luật nào cả. Làm sao mà để tất cả các phân số trên đời có chung mẫu số được? Vô lí. Rồi một người tiến một người lùi, cũng không xong, nếu muốn gần nhau hơn, chỉ có là cùng nhau hiểu về đối phương. Cái hiểu này không chỉ hiểu đối phương ở hiện tại, mà còn hiểu ở quá khứ. Từ đó mà chia sẻ và thấu hiểu. '''

In [3]:
summarizer = CentroidBOWSummarizer(language='english')
print('****** Summary ******\n')
print(summarizer.summarize(viet_in_2, limit=10))
print('\n\n\n ****** Original text ******')
print(viet_in_2)

****** Summary ******

Rồi một người tiến một người lùi, cũng không xong, nếu muốn gần nhau hơn, chỉ có là cùng nhau hiểu về đối phương.



 ****** Original text ******
Hồi còn nhỏ, ngây ngô nghĩ rằng, một mối quan hệ, nếu bị khoảng cách chia xa, thể nào cũng sẽ tàn phai. Thế nhưng nếu không xa nhau, thì tình cảm cũng sẽ còn mãi. Thế là hồi đó, chia tay bạn bè cấp 2, tôi buồn hết 2 năm cấp 3. Cảm giác của một đứa không có gì bám víu giữa thành phố xa lạ nhưng lại không chịu chấp nhận một ai xung quanh. Riết rồi khó chia sẻ đến tận bây giờ. Vốn tôi nghĩ, thật tình những vấn đề của bản thân, chẳng ai hiểu nổi. Một đứa trẻ 15 tuổi, bảo là vấn đề to lớn như phá sản, tang tóc,... thật không phải. Nhưng vấn đề có nhỏ không? Đáp án là không! Thay đổi môi trường sống thật ra chỉ là giọt nước tràn li cho rất nhiều những vấn đề khác nhau, khó có lối ra, chỉ toàn nút thắt. Có chăng chỉ là giống sợi chỉ đỏ, khâu mà có nút chỗ nào, thì đành cắt đi khâu lại ở nút đó. Cứ thế bản thân vừa muốn kết nối

In [4]:
eng_in

'The young boy who was hunting rabbits in the forest was not sure whether it had been the woman’s last cry or the child’s first that alerted him. He turned suddenly, sensing the possible danger, his eyes searching for an animal that was so obviously in pain. He had never known any animal to scream in quite that way before. He edged towards the noise cautiously; the scream had now turned to a whine but it still did not sound like any animal he knew. He hoped it would be small enough to kill; at least that would make a change from rabbit for dinner. The young boy moved stealthily towards the river, where the strange noise came from, running from tree to tree, feeling the protection of the bark against his shoulder blades, something to touch. Never stay in the open, his father had taught him. When he reached the edge of the forest, he had a clear line of vision all the way down the valley to the river, and even then it took him some time to realise that the strange cry emanated from no or